In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

In [3]:
df = pd.read_csv("../data/clean/df_after_EDA.csv")
df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [4]:
#taking a sample of data which has equal destrubution of tatget column values 
sample_df = df.groupby('readmitted').apply(lambda x: x.sample(n=100,random_state=2,replace=True))

/var/folders/q1/bq_b0f_d53ldgrm4rh3vh1_40000gn/T/ipykernel_41888/3690471984.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = df.groupby('readmitted').apply(lambda x: x.sample(n=100,random_state=2,replace=True))


In [5]:
#creating a new df with the remaining data
remaining_df = pd.concat([df,sample_df ]).drop_duplicates(keep=False)

In [6]:
print(f'{df.shape}')
print(f'{sample_df.shape}')
print(f'{remaining_df.shape}')

(882, 25)
(300, 25)
(637, 25)


In [7]:
# For training data
X_train = sample_df.drop(columns=['readmitted'])
y_train = sample_df['readmitted']

# For testing data
X_test = remaining_df.drop(columns=['readmitted'])
y_test = remaining_df['readmitted']

In [8]:
X_train_cat = X_train.select_dtypes('object')
X_test_cat  = X_test.select_dtypes('object')
#
X_train_num = X_train.select_dtypes('number')
X_test_num  = X_test.select_dtypes('number')

In [9]:
categorical_column_values = [ df[col].unique() for col in df.select_dtypes('object').columns ]

In [10]:
encoder = OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False)

In [27]:
encoder.fit(X_train_cat)
with open('../transformers/transformer_f.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [28]:
X_train_cat_encoded_np = encoder.transform(X_train_cat)
X_test_cat_encoded_np  = encoder.transform(X_test_cat)

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [5, 6, 7, 8, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [13]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

In [14]:
X_train_encoded = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test_encoded = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [29]:
normalizer = MinMaxScaler()
normalizer.fit(X_train_encoded)

with open('../scalers/scaler_f.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [16]:
X_train_norm_np = normalizer.transform(X_train_encoded)
X_test_norm_np = normalizer.transform(X_test_encoded)

In [17]:
X_train_norm_df = pd.DataFrame(X_train_norm_np, columns=X_train_encoded.columns, index=X_train_encoded.index)
X_test_norm_df  = pd.DataFrame(X_test_norm_np,  columns=X_test_encoded.columns,  index=X_test_encoded.index)

In [30]:
knn = KNeighborsClassifier(n_neighbors=5)  
knn.fit(X_train_norm_df, y_train)

with open('../models/f_KNN.pkl', 'wb') as f:
    pickle.dump(knn, f)


In [19]:
y_pred = knn.predict(X_test_norm_df)

In [20]:
accuracy = knn.score(X_test_norm_df, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.31711145996860285


In [31]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_norm_df, y_train)

with open('../models/f_DT.pkl', 'wb') as d:
    pickle.dump(dt_classifier, d)

In [22]:
dt_predictions = dt_classifier.predict(X_test_norm_df)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)

Decision Tree Accuracy: 0.40188383045525905


In [32]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_norm_df, y_train)

with open('../models/f_RF.pkl', 'wb') as r:
    pickle.dump(rf_classifier, r)

In [24]:
rf_predictions = rf_classifier.predict(X_test_norm_df)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.4489795918367347


In [25]:
dt_predictions = dt_classifier.predict(X_test_norm_df)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)

Decision Tree Accuracy: 0.40188383045525905


In [26]:
rf_predictions = rf_classifier.predict(X_test_norm_df)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.4489795918367347
